In [1]:
import requests
from bs4 import BeautifulSoup

In [2]:
import pandas as pd
import json

In [3]:
#link = "https://understat.com/match/22284"
res = requests.get("https://understat.com/player/1228")
soup = BeautifulSoup(res.content,'lxml')
scripts = soup.find_all('script')

In [4]:
strings = scripts[1].string 
ind_start = strings.index("('")+2 
ind_end = strings.index("')") 
json_data = strings[ind_start:ind_end] 
json_data = json_data.encode('utf8').decode('unicode_escape')
data = json.loads(json_data)

In [5]:
season_data = pd.json_normalize(data['season'])
season_data

,position,games,goals,shots,time,xG,assists,xA,key_passes,season,team,yellow,red,npg,npxG,xGChain,xGBuildup
0,MC,26,3,66,2340,6.373083735816181,6,8.412570178508759,76,2023,Manchester United,8,0,2,4.850745971314609,15.270424462854862,8.927047699689865
1,MC,37,8,92,3326,9.385456696152687,8,17.266709440387785,118,2022,Manchester United,6,0,6,7.863118812441826,35.418096881359816,17.724953144788742
2,AMC,36,10,89,3135,9.285304862074554,6,8.811852657236159,88,2021,Manchester United,10,0,10,7.762967492453754,22.98672279715538,11.14225958287716
3,AMC,37,18,121,3117,16.019450719468296,12,11.474997889250517,95,2020,Manchester United,6,0,9,8.407837779261172,26.91141940653324,11.932286506518722
4,AMC,14,8,44,1211,5.98607742600143,7,2.7549998983740807,30,2019,Manchester United,2,0,4,2.9414413776248693,8.513753656297922,4.864240288734436
5,Sub,33,5,62,1736,5.761577281169593,2,4.4775946363806725,52,2016,Sampdoria,2,0,5,5.761577281169593,11.588001352734864,3.9690468413755298
6,MC,31,3,52,2116,3.79332700651139,4,3.7490005446597934,44,2015,Udinese,7,1,2,2.2707290379330516,8.635709737427533,4.768451112322509
7,Sub,31,3,41,1605,2.634341513738036,2,2.331952504813671,22,2014,Udinese,5,0,3,2.634341513738036,5.787391685880721,2.2256615795195103


In [6]:
position_data = pd.json_normalize(data['position'])
position_data.T

,0
2023.AMC.position,AMC
2023.AMC.games,21
2023.AMC.goals,2
2023.AMC.shots,55
2023.AMC.time,1890
...,...
2014.ML.red,0
2014.ML.npg,0
2014.ML.npxG,0.2823210060596466
2014.ML.xGChain,1.0028970018029213


In [7]:
position_2023 = pd.json_normalize(data['position']['2023'])
position_2023.T

,0
AMC.position,AMC
AMC.games,21
AMC.goals,2
AMC.shots,55
AMC.time,1890
...,...
ML.red,0
ML.npg,0
ML.npxG,0
ML.xGChain,0.15376435220241547


In [8]:
position_2023_AMC = pd.json_normalize(data['position']['2023']['AMC'])
position_2023_AMR = pd.json_normalize(data['position']['2023']['AMR'])
position_2023_MC = pd.json_normalize(data['position']['2023']['MC'])
positions_2023 = pd.concat([position_2023_AMC,position_2023_AMR,
                            position_2023_MC])
positions_2023[['games','goals','shots','time','xG','assists','xA',
                'key_passes','yellow','red','npg','npxG','xGChain',
                'xGBuildup']] = positions_2023[['games','goals',
                'shots','time','xG','assists','xA',
                'key_passes','yellow','red','npg','npxG',
                'xGChain','xGBuildup']].apply(pd.to_numeric)
positions_2023.loc['total']= positions_2023.sum()
positions_2023.loc[positions_2023.index[-1], 'position'] = ''
positions_2023.loc[positions_2023.index[-1], 'season'] = '2023'
positions_2023
#on the website/page, there's also Sh90 (shots per 90), KP90(key passes per90),
#xG90 (xG per 90), xA90 (xA per 90), but not in the raw json data -> it could
#calculated by getting the 90s from the 'time' column (value/90) and creating
#new columns for each metric where we divide the total values by the 90s
#they also show next to expected metrics the difference compared to actual 
#values (goals, assists), but I'm not replicating it for now

,position,games,goals,shots,time,xG,assists,xA,key_passes,season,yellow,red,npg,npxG,xGChain,xGBuildup
0,AMC,21,2,55,1890,5.723903,5,7.684727,66,2023,7,0,1,4.201565,12.649727,7.371080
0,AMR,3,1,10,270,0.632634,0,0.265181,5,2023,1,0,1,0.632634,1.394165,0.706289
0,MC,1,0,1,90,0.016548,0,0.308898,3,2023,0,0,0,0.016548,1.072768,0.849679
total,,25,3,66,2250,6.373084,5,8.258806,74,2023,8,0,2,4.850746,15.116660,8.927048


In [9]:
position_2022_AMC = pd.json_normalize(data['position']['2022']['AMC'])
position_2022_AMR = pd.json_normalize(data['position']['2022']['AMR'])
position_2022_AML = pd.json_normalize(data['position']['2022']['AML'])
position_2022_MC = pd.json_normalize(data['position']['2022']['MC'])
position_2022_DMC = pd.json_normalize(data['position']['2022']['DMC'])
position_2022_MR = pd.json_normalize(data['position']['2022']['MR'])
positions_2022 = pd.concat([position_2022_AMC,position_2022_AMR,
                            position_2022_AML,position_2022_DMC,
                            position_2022_MC,position_2022_MR])
positions_2022[['games','goals','shots','time','xG','assists','xA',
                'key_passes','yellow','red','npg','npxG','xGChain',
                'xGBuildup']] = positions_2022[['games','goals',
                'shots','time','xG','assists','xA',
                'key_passes','yellow','red','npg','npxG',
                'xGChain','xGBuildup']].apply(pd.to_numeric)
position_2021_AMC = pd.json_normalize(data['position']['2021']['AMC'])
position_2021_AML = pd.json_normalize(data['position']['2021']['AML'])
position_2021_MC = pd.json_normalize(data['position']['2021']['MC'])
position_2021_Sub = pd.json_normalize(data['position']['2021']['Sub'])
position_2021_FW = pd.json_normalize(data['position']['2021']['FW'])
positions_2021 = pd.concat([position_2021_AMC,position_2021_AML,
                            position_2021_MC,position_2021_Sub,
                            position_2021_FW])
positions_2021[['games','goals','shots','time','xG','assists','xA',
                'key_passes','yellow','red','npg','npxG','xGChain',
                'xGBuildup']] = positions_2021[['games','goals',
                'shots','time','xG','assists','xA',
                'key_passes','yellow','red','npg','npxG',
                'xGChain','xGBuildup']].apply(pd.to_numeric)
position_2020_AMC = pd.json_normalize(data['position']['2020']['AMC'])
position_2020_Sub = pd.json_normalize(data['position']['2020']['Sub'])
positions_2020 = pd.concat([position_2020_AMC,position_2020_Sub])
positions_2020[['games','goals','shots','time','xG','assists','xA',
                'key_passes','yellow','red','npg','npxG','xGChain',
                'xGBuildup']] = positions_2020[['games','goals',
                'shots','time','xG','assists','xA',
                'key_passes','yellow','red','npg','npxG',
                'xGChain','xGBuildup']].apply(pd.to_numeric)
positions_2019 = pd.json_normalize(data['position']['2019']['AMC'])
positions_2019[['games','goals','shots','time','xG','assists','xA',
                'key_passes','yellow','red','npg','npxG','xGChain',
                'xGBuildup']] = positions_2019[['games','goals',
                'shots','time','xG','assists','xA',
                'key_passes','yellow','red','npg','npxG',
                'xGChain','xGBuildup']].apply(pd.to_numeric)
position_2016_AMC = pd.json_normalize(data['position']['2016']['AMC'])
position_2016_Sub = pd.json_normalize(data['position']['2016']['Sub'])
positions_2016 = pd.concat([position_2016_AMC,position_2016_Sub])
positions_2016[['games','goals','shots','time','xG','assists','xA',
                'key_passes','yellow','red','npg','npxG','xGChain',
                'xGBuildup']] = positions_2016[['games','goals',
                'shots','time','xG','assists','xA',
                'key_passes','yellow','red','npg','npxG',
                'xGChain','xGBuildup']].apply(pd.to_numeric)
position_2015_AMC = pd.json_normalize(data['position']['2015']['AMC'])
position_2015_MC = pd.json_normalize(data['position']['2015']['MC'])
position_2015_Sub = pd.json_normalize(data['position']['2015']['Sub'])
position_2015_FWL = pd.json_normalize(data['position']['2015']['FWL'])
position_2015_FWR = pd.json_normalize(data['position']['2015']['FWR'])
positions_2015 = pd.concat([position_2015_AMC,position_2015_MC,
                            position_2015_Sub,position_2015_FWL,
                            position_2015_FWR])
positions_2015[['games','goals','shots','time','xG','assists','xA',
                'key_passes','yellow','red','npg','npxG','xGChain',
                'xGBuildup']] = positions_2015[['games','goals',
                'shots','time','xG','assists','xA',
                'key_passes','yellow','red','npg','npxG',
                'xGChain','xGBuildup']].apply(pd.to_numeric)
position_2014_AMC = pd.json_normalize(data['position']['2014']['AMC'])
position_2014_MC = pd.json_normalize(data['position']['2014']['MC'])
position_2014_Sub = pd.json_normalize(data['position']['2014']['Sub'])
position_2014_ML = pd.json_normalize(data['position']['2014']['ML'])
positions_2014 = pd.concat([position_2014_AMC,position_2014_MC,
                            position_2014_Sub,position_2014_ML])
positions_2014[['games','goals','shots','time','xG','assists','xA',
                'key_passes','yellow','red','npg','npxG','xGChain',
                'xGBuildup']] = positions_2014[['games','goals',
                'shots','time','xG','assists','xA',
                'key_passes','yellow','red','npg','npxG',
                'xGChain','xGBuildup']].apply(pd.to_numeric)

In [11]:
positions_all = pd.concat([positions_2014,positions_2015,positions_2016,positions_2019,
                           positions_2020,positions_2021,positions_2022,positions_2023])
positions_all = positions_all.groupby('position').sum()
#positions_all.drop('AMCAMRMC',inplace=True)
positions_all.drop(['season'],axis=1,inplace=True)
positions_all.loc['total']= positions_all.sum()
positions_all.loc[positions_all.index[-1], 'position'] = ''
positions_all

,games,goals,shots,time,xG,assists,xA,key_passes,yellow,red,npg,npxG,xGChain,xGBuildup,position
position,,,,,,,,,,,,,,,
,25.0,3.0,66.0,2250.0,6.373084,5.0,8.258806,74.0,8.0,0.0,2.0,4.850746,15.116660,8.927048,NaN
AMC,158.0,46.0,417.0,13368.0,46.330478,37.0,44.410738,393.0,35.0,0.0,30.0,31.107216,100.064836,50.135219,NaN
AML,5.0,1.0,20.0,450.0,2.042482,0.0,1.484221,11.0,0.0,0.0,1.0,2.042482,3.942079,1.622461,NaN
AMR,6.0,2.0,18.0,540.0,1.328287,3.0,3.652121,19.0,1.0,0.0,2.0,1.328287,7.068355,2.471354,NaN
DMC,2.0,0.0,5.0,180.0,0.443033,0.0,0.976870,9.0,1.0,0.0,0.0,0.443033,2.031524,1.457587,NaN
FW,2.0,0.0,5.0,180.0,0.204547,0.0,0.117889,2.0,0.0,0.0,0.0,0.204547,0.498258,0.268235,NaN
FWL,1.0,1.0,3.0,90.0,0.373913,0.0,0.132882,4.0,0.0,0.0,1.0,0.373913,0.484263,0.051761,NaN
FWR,2.0,0.0,2.0,171.0,0.028145,0.0,0.152807,3.0,0.0,0.0,0.0,0.028145,0.580428,0.447162,NaN
MC,28.0,3.0,56.0,2336.0,3.812721,5.0,4.827663,50.0,7.0,1.0,2.0,2.290123,9.458679,4.708360,NaN


In [12]:
df_position_table = pd.DataFrame(data['position'])
df_position_table

,2023,2022,2021,2020,2019,2016,2015,2014
AMC,"{'position': 'AMC', 'games': '21', 'goals': '2...","{'position': 'AMC', 'games': '26', 'goals': '6...","{'position': 'AMC', 'games': '28', 'goals': '9...","{'position': 'AMC', 'games': '35', 'goals': '1...","{'position': 'AMC', 'games': '14', 'goals': '8...","{'position': 'AMC', 'games': '22', 'goals': '2...","{'position': 'AMC', 'games': '3', 'goals': '0'...","{'position': 'AMC', 'games': '9', 'goals': '1'..."
AMR,"{'position': 'AMR', 'games': '3', 'goals': '1'...","{'position': 'AMR', 'games': '3', 'goals': '1'...",NaN,NaN,NaN,NaN,NaN,NaN
MC,"{'position': 'MC', 'games': '1', 'goals': '0',...","{'position': 'MC', 'games': '3', 'goals': '0',...","{'position': 'MC', 'games': '2', 'goals': '0',...",NaN,NaN,NaN,"{'position': 'MC', 'games': '18', 'goals': '2'...","{'position': 'MC', 'games': '4', 'goals': '1',..."
ML,"{'position': 'ML', 'games': '1', 'goals': '0',...",NaN,NaN,NaN,NaN,NaN,NaN,"{'position': 'ML', 'games': '3', 'goals': '0',..."
AML,NaN,"{'position': 'AML', 'games': '2', 'goals': '0'...","{'position': 'AML', 'games': '3', 'goals': '1'...",NaN,NaN,NaN,NaN,NaN
DMC,NaN,"{'position': 'DMC', 'games': '2', 'goals': '0'...",NaN,NaN,NaN,NaN,NaN,NaN
MR,NaN,"{'position': 'MR', 'games': '1', 'goals': '1',...",NaN,NaN,NaN,NaN,NaN,NaN
FW,NaN,NaN,"{'position': 'FW', 'games': '2', 'goals': '0',...",NaN,NaN,NaN,NaN,NaN
Sub,NaN,NaN,"{'position': 'Sub', 'games': '1', 'goals': '0'...","{'position': 'Sub', 'games': '2', 'goals': '0'...",NaN,"{'position': 'Sub', 'games': '11', 'goals': '3...","{'position': 'Sub', 'games': '7', 'goals': '0'...","{'position': 'Sub', 'games': '15', 'goals': '1..."
FWR,NaN,NaN,NaN,NaN,NaN,NaN,"{'position': 'FWR', 'games': '2', 'goals': '0'...",NaN


In [13]:
df_position_table['2022'].apply(pd.Series)

,position,games,goals,shots,time,xG,assists,xA,key_passes,season,yellow,red,npg,npxG,xGChain,xGBuildup,0
AMC,AMC,26,6,67,2336,7.438450904563069,5,10.198626049794257,79,2022,5,0,4,5.916113020852208,23.945894669741392,13.256883010268211,NaN
AMR,AMR,3,1,8,270,0.6956530064344406,3,3.3869400024414062,14,2022,0,0,1,0.6956530064344406,5.674190044403076,1.7650649845600128,NaN
MC,MC,3,0,5,270,0.23019780963659286,0,1.5511826574802399,9,2022,0,0,0,0.23019780963659286,1.8387940227985382,0.5520040616393089,NaN
ML,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
AML,AML,2,0,5,180,0.1914479210972786,0,0.8254640996456146,6,2022,0,0,0,0.1914479210972786,1.0212895572185516,0.5013101175427437,NaN
DMC,DMC,2,0,5,180,0.4430330079048872,0,0.976869985461235,9,2022,1,0,0,0.4430330079048872,2.03152397274971,1.4575870335102081,NaN
MR,MR,1,1,2,90,0.38667404651641846,0,0.32762664556503296,1,2022,0,0,1,0.38667404651641846,0.9064046144485474,0.19210393726825714,NaN
FW,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Sub,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
FWR,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
pd.DataFrame(data['position']['2021']).T

,position,games,goals,shots,time,xG,assists,xA,key_passes,season,yellow,red,npg,npxG,xGChain,xGBuildup
AMC,AMC,28,9,63,2476,6.694973829202354,6,7.842104258947074,79,2021,10,0,9,5.172636459581554,18.25581879168749,9.00605158507824
AML,AML,3,1,15,270,1.8510341122746468,0,0.6587570011615753,5,2021,0,0,1,1.8510341122746468,2.9207890033721924,1.1211509704589844
MC,MC,2,0,4,180,0.11685980297625065,0,0.09007179737091064,1,2021,0,0,0,0.11685980297625065,0.7768729999661446,0.641677625477314
FW,FW,2,0,5,180,0.2045471053570509,0,0.11788859777152538,2,2021,0,0,0,0.2045471053570509,0.4982580095529556,0.2682354003190994
Sub,Sub,1,0,2,29,0.4178900122642517,0,0.10303100198507309,1,2021,0,0,0,0.4178900122642517,0.5349839925765991,0.10514400154352188


In [15]:
pd.DataFrame(data['situation']['2022']).T

,situation,season,goals,shots,xG,assists,key_passes,xA,npg,npxG,time
OpenPlay,OpenPlay,2022,6,79,7.253958882763982,8,88,14.336424613371491,6,7.253958882763982,3326
DirectFreekick,DirectFreekick,2022,0,6,0.4032558798789978,0,0,0,0,0.4032558798789978,3326
FromCorner,FromCorner,2022,0,5,0.2059037685394287,0,27,2.0791266644373536,0,0.2059037685394287,3326
Penalty,Penalty,2022,2,2,1.5223378539085388,0,0,0,0,0,3326
SetPiece,SetPiece,2022,0,0,0,0,3,0.8511593155562878,0,0,3326


In [16]:
pd.DataFrame(data['shotZones']['2022']).T

,shotZones,season,goals,shots,xG,assists,key_passes,xA,npg,npxG
shotOboxTotal,shotOboxTotal,2022,1,47,1.810519153252244,2,36,1.3159849112853408,1,1.810519153252244
shotPenaltyArea,shotPenaltyArea,2022,6,42,6.459120161831379,5,72,11.47144291549921,4,4.93678230792284
shotSixYardBox,shotSixYardBox,2022,1,3,1.1158170700073242,1,10,4.479282766580582,1,1.1158170700073242


In [17]:
pd.DataFrame(data['shotTypes']['2022']).T

,shotTypes,season,goals,shots,xG,assists,key_passes,xA,npg,npxG
RightFoot,RightFoot,2022,8,73,8.125196631997824,5,51,7.224511770531535,6,6.602858778089285
LeftFoot,LeftFoot,2022,0,11,0.4663883689790964,3,53,8.721979978494346,0,0.4663883689790964
Head,Head,2022,0,8,0.793871384114027,0,14,1.3202188443392515,0,0.793871384114027


In [18]:
openplay_2022 = pd.json_normalize(data['situation']['2022']['OpenPlay'])
freekick_2022 = pd.json_normalize(data['situation']['2022']['DirectFreekick'])
corner_2022 = pd.json_normalize(data['situation']['2022']['FromCorner'])
penalty_2022 = pd.json_normalize(data['situation']['2022']['Penalty'])
setpiece_2022 = pd.json_normalize(data['situation']['2022']['SetPiece'])
situation_2022 = pd.concat([openplay_2022,freekick_2022,
                            corner_2022,penalty_2022,
                            setpiece_2022,setpiece_2022])
situation_2022[['goals','shots','xG','assists','xA',
                'key_passes','npg','npxG']] = situation_2022[['goals','shots','xG','assists','xA',
                'key_passes','npg','npxG']].apply(pd.to_numeric)
situation_2022.loc['total']= situation_2022.sum()
situation_2022.loc[situation_2022.index[-1], 'situation'] = ''
situation_2022.loc[situation_2022.index[-1], 'time'] = ''
situation_2022.loc[situation_2022.index[-1], 'season'] = '2022'
situation_2022

,situation,season,goals,shots,xG,assists,key_passes,xA,npg,npxG,time
0,OpenPlay,2022,6,79,7.253959,8,88,14.336425,6,7.253959,3326
0,DirectFreekick,2022,0,6,0.403256,0,0,0.000000,0,0.403256,3326
0,FromCorner,2022,0,5,0.205904,0,27,2.079127,0,0.205904,3326
0,Penalty,2022,2,2,1.522338,0,0,0.000000,0,0.000000,3326
0,SetPiece,2022,0,0,0.000000,0,3,0.851159,0,0.000000,3326
0,SetPiece,2022,0,0,0.000000,0,3,0.851159,0,0.000000,3326
total,,2022,8,92,9.385456,8,121,18.117870,6,7.863119,


In [19]:
openplay_2023 = pd.json_normalize(data['situation']['2023']['OpenPlay'])
freekick_2023 = pd.json_normalize(data['situation']['2023']['DirectFreekick'])
corner_2023 = pd.json_normalize(data['situation']['2023']['FromCorner'])
penalty_2023 = pd.json_normalize(data['situation']['2023']['Penalty'])
setpiece_2023 = pd.json_normalize(data['situation']['2023']['SetPiece'])
situation_2023 = pd.concat([openplay_2023,freekick_2023,
                            corner_2023,penalty_2023,
                            setpiece_2023,setpiece_2023])
openplay_2022 = pd.json_normalize(data['situation']['2022']['OpenPlay'])
freekick_2022 = pd.json_normalize(data['situation']['2022']['DirectFreekick'])
corner_2022 = pd.json_normalize(data['situation']['2022']['FromCorner'])
penalty_2022 = pd.json_normalize(data['situation']['2022']['Penalty'])
setpiece_2022 = pd.json_normalize(data['situation']['2022']['SetPiece'])
situation_2022 = pd.concat([openplay_2022,freekick_2022,
                            corner_2022,penalty_2022,
                            setpiece_2022,setpiece_2022])
openplay_2021 = pd.json_normalize(data['situation']['2021']['OpenPlay'])
freekick_2021 = pd.json_normalize(data['situation']['2021']['DirectFreekick'])
corner_2021 = pd.json_normalize(data['situation']['2021']['FromCorner'])
penalty_2021 = pd.json_normalize(data['situation']['2021']['Penalty'])
setpiece_2021 = pd.json_normalize(data['situation']['2021']['SetPiece'])
situation_2021 = pd.concat([openplay_2021,freekick_2021,
                            corner_2021,penalty_2021,
                            setpiece_2021,setpiece_2021])

openplay_2020 = pd.json_normalize(data['situation']['2020']['OpenPlay'])
freekick_2020 = pd.json_normalize(data['situation']['2020']['DirectFreekick'])
corner_2020 = pd.json_normalize(data['situation']['2020']['FromCorner'])
penalty_2020 = pd.json_normalize(data['situation']['2020']['Penalty'])
setpiece_2020 = pd.json_normalize(data['situation']['2020']['SetPiece'])
situation_2020 = pd.concat([openplay_2020,freekick_2020,
                            corner_2020,penalty_2020,
                            setpiece_2020,setpiece_2020])

openplay_2019 = pd.json_normalize(data['situation']['2019']['OpenPlay'])
freekick_2019 = pd.json_normalize(data['situation']['2019']['DirectFreekick'])
corner_2019 = pd.json_normalize(data['situation']['2019']['FromCorner'])
penalty_2019 = pd.json_normalize(data['situation']['2019']['Penalty'])
setpiece_2019 = pd.json_normalize(data['situation']['2019']['SetPiece'])
situation_2019 = pd.concat([openplay_2019,freekick_2019,
                            corner_2019,penalty_2019,
                            setpiece_2019,setpiece_2019])

openplay_2016 = pd.json_normalize(data['situation']['2016']['OpenPlay'])
freekick_2016 = pd.json_normalize(data['situation']['2016']['DirectFreekick'])
corner_2016 = pd.json_normalize(data['situation']['2016']['FromCorner'])
setpiece_2016 = pd.json_normalize(data['situation']['2016']['SetPiece'])
situation_2016 = pd.concat([openplay_2016,freekick_2016,
                            corner_2016,
                            setpiece_2016,setpiece_2016])
openplay_2015 = pd.json_normalize(data['situation']['2015']['OpenPlay'])
freekick_2015 = pd.json_normalize(data['situation']['2015']['DirectFreekick'])
corner_2015 = pd.json_normalize(data['situation']['2015']['FromCorner'])
penalty_2015 = pd.json_normalize(data['situation']['2015']['Penalty'])
setpiece_2015 = pd.json_normalize(data['situation']['2015']['SetPiece'])
situation_2015 = pd.concat([openplay_2015,freekick_2015,
                            corner_2015,penalty_2015,
                            setpiece_2015,setpiece_2015])
openplay_2014 = pd.json_normalize(data['situation']['2014']['OpenPlay'])
freekick_2014 = pd.json_normalize(data['situation']['2014']['DirectFreekick'])
corner_2014 = pd.json_normalize(data['situation']['2014']['FromCorner'])
situation_2014 = pd.concat([openplay_2014,freekick_2014,
                            corner_2014])


situation_all = pd.concat([situation_2014,situation_2015,situation_2016,situation_2019,
                           situation_2020,situation_2021,situation_2022,situation_2023])
situation_all[['goals','shots','xG','assists','xA',
                'key_passes','npg','npxG']] = situation_all[['goals','shots','xG','assists','xA',
                'key_passes','npg','npxG']].apply(pd.to_numeric)
situation_all.drop(['season'],axis=1,inplace=True)
situation_all.drop(['time'],axis=1,inplace=True)
situation_all = situation_all.groupby('situation').sum()
situation_all.loc['total']= situation_all.sum()
situation_all

,goals,shots,xG,assists,key_passes,xA,npg,npxG
situation,,,,,,,,
DirectFreekick,1.0,57.0,3.656317,0.0,0.0,0.000000,1.0,3.656317
FromCorner,0.0,45.0,1.129803,5.0,111.0,8.295919,0.0,1.129803
OpenPlay,38.0,436.0,36.908615,40.0,387.0,46.787052,38.0,36.908615
Penalty,17.0,22.0,16.745867,0.0,0.0,0.000000,0.0,0.000000
SetPiece,4.0,14.0,1.596062,4.0,52.0,8.393415,4.0,1.596062
total,60.0,574.0,60.036663,49.0,550.0,63.476386,43.0,43.290796


In [20]:
shotPenaltyArea_2022 = pd.json_normalize(data['shotZones']['2022']['shotPenaltyArea'])
shotOboxTotal_2022 = pd.json_normalize(data['shotZones']['2022']['shotOboxTotal'])
shotSixYardBox_2022 = pd.json_normalize(data['shotZones']['2022']['shotSixYardBox'])
shotzones_2022 = pd.concat([shotPenaltyArea_2022,shotOboxTotal_2022,
                            shotSixYardBox_2022])
shotzones_2022[['goals','shots','xG','assists','xA',
                'key_passes','npg','npxG']] = shotzones_2022[['goals','shots','xG','assists','xA',
                'key_passes','npg','npxG']].apply(pd.to_numeric)
shotzones_2022.loc['total']= shotzones_2022.sum()
shotzones_2022.loc[shotzones_2022.index[-1], 'shotZones'] = ''

shotzones_2022.loc[shotzones_2022.index[-1], 'season'] = '2022'
shotzones_2022
#on the website there are also xG/Sh and xA/KP columns -> these could be calculated from this data...

,shotZones,season,goals,shots,xG,assists,key_passes,xA,npg,npxG
0,shotPenaltyArea,2022,6,42,6.459120,5,72,11.471443,4,4.936782
0,shotOboxTotal,2022,1,47,1.810519,2,36,1.315985,1,1.810519
0,shotSixYardBox,2022,1,3,1.115817,1,10,4.479283,1,1.115817
total,,2022,8,92,9.385456,8,118,17.266711,6,7.863119


In [21]:
shotPenaltyArea_2023 = pd.json_normalize(data['shotZones']['2023']['shotPenaltyArea'])
shotOboxTotal_2023 = pd.json_normalize(data['shotZones']['2023']['shotOboxTotal'])
shotSixYardBox_2023 = pd.json_normalize(data['shotZones']['2023']['shotSixYardBox'])
shotzones_2023 = pd.concat([shotPenaltyArea_2023,shotOboxTotal_2023,
                            shotSixYardBox_2023])

shotPenaltyArea_2022 = pd.json_normalize(data['shotZones']['2022']['shotPenaltyArea'])
shotOboxTotal_2022 = pd.json_normalize(data['shotZones']['2022']['shotOboxTotal'])
shotSixYardBox_2022 = pd.json_normalize(data['shotZones']['2022']['shotSixYardBox'])
shotzones_2022 = pd.concat([shotPenaltyArea_2022,shotOboxTotal_2022,
                            shotSixYardBox_2022])

shotPenaltyArea_2021 = pd.json_normalize(data['shotZones']['2021']['shotPenaltyArea'])
shotOboxTotal_2021 = pd.json_normalize(data['shotZones']['2021']['shotOboxTotal'])
shotSixYardBox_2021 = pd.json_normalize(data['shotZones']['2021']['shotSixYardBox'])
shotzones_2021 = pd.concat([shotPenaltyArea_2021,shotOboxTotal_2021,
                            shotSixYardBox_2021])

shotPenaltyArea_2020 = pd.json_normalize(data['shotZones']['2020']['shotPenaltyArea'])
shotOboxTotal_2020 = pd.json_normalize(data['shotZones']['2020']['shotOboxTotal'])
shotSixYardBox_2020 = pd.json_normalize(data['shotZones']['2020']['shotSixYardBox'])
shotzones_2020 = pd.concat([shotPenaltyArea_2020,shotOboxTotal_2020,
                            shotSixYardBox_2020])

shotPenaltyArea_2019 = pd.json_normalize(data['shotZones']['2019']['shotPenaltyArea'])
shotOboxTotal_2019 = pd.json_normalize(data['shotZones']['2019']['shotOboxTotal'])
shotzones_2019 = pd.concat([shotPenaltyArea_2019,shotOboxTotal_2019])

shotPenaltyArea_2016 = pd.json_normalize(data['shotZones']['2016']['shotPenaltyArea'])
shotOboxTotal_2016 = pd.json_normalize(data['shotZones']['2016']['shotOboxTotal'])
shotSixYardBox_2016 = pd.json_normalize(data['shotZones']['2016']['shotSixYardBox'])
shotzones_2016 = pd.concat([shotPenaltyArea_2016,shotOboxTotal_2016,
                            shotSixYardBox_2016])

shotPenaltyArea_2015 = pd.json_normalize(data['shotZones']['2015']['shotPenaltyArea'])
shotOboxTotal_2015 = pd.json_normalize(data['shotZones']['2015']['shotOboxTotal'])
shotSixYardBox_2015 = pd.json_normalize(data['shotZones']['2015']['shotSixYardBox'])
shotzones_2015 = pd.concat([shotPenaltyArea_2015,shotOboxTotal_2015,
                            shotSixYardBox_2015])

shotPenaltyArea_2014 = pd.json_normalize(data['shotZones']['2014']['shotPenaltyArea'])
shotOboxTotal_2014 = pd.json_normalize(data['shotZones']['2014']['shotOboxTotal'])
shotSixYardBox_2014 = pd.json_normalize(data['shotZones']['2014']['shotSixYardBox'])
shotzones_2014 = pd.concat([shotPenaltyArea_2014,shotOboxTotal_2014,
                            shotSixYardBox_2014])

shotzones_all = pd.concat([shotzones_2014,shotzones_2015,shotzones_2016,shotzones_2019,
                           shotzones_2020,shotzones_2021,shotzones_2022,shotzones_2023])
shotzones_all[['goals','shots','xG','assists','xA',
                'key_passes','npg','npxG']] = shotzones_all[['goals','shots','xG','assists','xA',
                'key_passes','npg','npxG']].apply(pd.to_numeric)
shotzones_all.drop(['season'],axis=1,inplace=True)
shotzones_all = shotzones_all.groupby('shotZones').sum()
shotzones_all.loc['total']= shotzones_all.sum()
shotzones_all

,goals,shots,xG,assists,key_passes,xA,npg,npxG
shotZones,,,,,,,,
shotOboxTotal,11.0,351.0,11.777870,9.0,152.0,4.432112,11.0,11.777870
shotPenaltyArea,45.0,205.0,43.174199,32.0,337.0,42.859953,28.0,26.428333
shotSixYardBox,2.0,11.0,4.286562,6.0,35.0,11.987613,2.0,4.286562
total,58.0,567.0,59.238632,47.0,524.0,59.279678,41.0,42.492765


In [22]:
right_2023 = pd.json_normalize(data['shotTypes']['2023']['RightFoot'])
left_2023 = pd.json_normalize(data['shotTypes']['2023']['LeftFoot'])
head_2023 = pd.json_normalize(data['shotTypes']['2023']['Head'])
shottypes_2023 = pd.concat([right_2023,left_2023,head_2023])
shottypes_2023[['goals','shots','xG','assists','xA',
                'key_passes','npg','npxG']] = shottypes_2023[['goals','shots','xG','assists','xA',
                'key_passes','npg','npxG']].apply(pd.to_numeric)
shottypes_2023.drop(['season'],axis=1,inplace=True)
shottypes_2023 = shottypes_2023.groupby('shotTypes').sum()
shottypes_2023.loc['total']= shottypes_2023.sum()
shottypes_2023

,goals,shots,xG,assists,key_passes,xA,npg,npxG
shotTypes,,,,,,,,
Head,0.0,2.0,0.479909,1.0,20.0,3.203741,0.0,0.479909
LeftFoot,0.0,10.0,0.465785,1.0,25.0,2.166760,0.0,0.465785
RightFoot,3.0,54.0,5.427389,4.0,31.0,3.042069,2.0,3.905051
total,3.0,66.0,6.373084,6.0,76.0,8.412570,2.0,4.850746


In [23]:
right_2023 = pd.json_normalize(data['shotTypes']['2023']['RightFoot'])
left_2023 = pd.json_normalize(data['shotTypes']['2023']['LeftFoot'])
head_2023 = pd.json_normalize(data['shotTypes']['2023']['Head'])
shottypes_2023 = pd.concat([right_2023,left_2023,head_2023])

right_2022 = pd.json_normalize(data['shotTypes']['2022']['RightFoot'])
left_2022 = pd.json_normalize(data['shotTypes']['2022']['LeftFoot'])
head_2022 = pd.json_normalize(data['shotTypes']['2022']['Head'])
shottypes_2022 = pd.concat([right_2022,left_2022,head_2022])

right_2021 = pd.json_normalize(data['shotTypes']['2021']['RightFoot'])
left_2021 = pd.json_normalize(data['shotTypes']['2021']['LeftFoot'])
head_2021 = pd.json_normalize(data['shotTypes']['2021']['Head'])
other_2021 = pd.json_normalize(data['shotTypes']['2021']['OtherBodyPart'])
shottypes_2021 = pd.concat([right_2021,left_2021,head_2021,other_2021])

right_2020 = pd.json_normalize(data['shotTypes']['2020']['RightFoot'])
left_2020 = pd.json_normalize(data['shotTypes']['2020']['LeftFoot'])
head_2020 = pd.json_normalize(data['shotTypes']['2020']['Head'])
shottypes_2020 = pd.concat([right_2020,left_2020,head_2020])

right_2019 = pd.json_normalize(data['shotTypes']['2019']['RightFoot'])
left_2019 = pd.json_normalize(data['shotTypes']['2019']['LeftFoot'])
head_2019 = pd.json_normalize(data['shotTypes']['2019']['Head'])
shottypes_2019 = pd.concat([right_2019,left_2019,head_2019])

right_2016 = pd.json_normalize(data['shotTypes']['2016']['RightFoot'])
left_2016 = pd.json_normalize(data['shotTypes']['2016']['LeftFoot'])
head_2016 = pd.json_normalize(data['shotTypes']['2016']['Head'])
shottypes_2016 = pd.concat([right_2016,left_2016,head_2016])

right_2015 = pd.json_normalize(data['shotTypes']['2015']['RightFoot'])
left_2015 = pd.json_normalize(data['shotTypes']['2015']['LeftFoot'])
head_2015 = pd.json_normalize(data['shotTypes']['2015']['Head'])
shottypes_2015 = pd.concat([right_2015,left_2015,head_2015])

right_2014 = pd.json_normalize(data['shotTypes']['2014']['RightFoot'])
left_2014 = pd.json_normalize(data['shotTypes']['2014']['LeftFoot'])
head_2014 = pd.json_normalize(data['shotTypes']['2014']['Head'])
shottypes_2014 = pd.concat([right_2014,left_2014,head_2014])

shottypes_all = pd.concat([shottypes_2014,shottypes_2015,shottypes_2016,shottypes_2019,
                           shottypes_2020,shottypes_2021,shottypes_2022,shottypes_2023])
shottypes_all[['goals','shots','xG','assists','xA',
                'key_passes','npg','npxG']] = shottypes_all[['goals','shots','xG','assists','xA',
                'key_passes','npg','npxG']].apply(pd.to_numeric)
shottypes_all.drop(['season'],axis=1,inplace=True)
shottypes_all = shottypes_all.groupby('shotTypes').sum()
shottypes_all.loc['total']= shottypes_all.sum()
shottypes_all

,goals,shots,xG,assists,key_passes,xA,npg,npxG
shotTypes,,,,,,,,
Head,2.0,30.0,3.836019,8.0,114.0,11.573813,2.0,3.836019
LeftFoot,4.0,62.0,4.108991,11.0,182.0,21.240115,4.0,4.108991
OtherBodyPart,0.0,0.0,0.000000,0.0,1.0,0.040029,0.0,0.000000
RightFoot,52.0,475.0,51.293622,28.0,227.0,26.425721,35.0,34.547756
total,58.0,567.0,59.238632,47.0,524.0,59.279678,41.0,42.492765


In [24]:
strings2 = scripts[2].string 
ind_start2 = strings2.index("('")+2 
ind_end2 = strings2.index("')") 
json_data2 = strings2[ind_start2:ind_end2] 
json_data2 = json_data2.encode('utf8').decode('unicode_escape')
data2 = json.loads(json_data2)

In [25]:
#note: this is the data I found within the "scripts", but it's not exactly the stuff you see in 
#the page (per 90 values, radar, seasons menu)
radar_amc = pd.json_normalize(data2['AMC'])
radar_amc.loc[:,'position'] = 'AMC'
radar_aml = pd.json_normalize(data2['AML'])
radar_aml.loc[:,'position'] = 'AML'
radar_amr = pd.json_normalize(data2['AMR'])
radar_amr.loc[:,'position'] = 'AMR'
radar_dmc = pd.json_normalize(data2['DMC'])
radar_dmc.loc[:,'position'] = 'DMC'
radar_fw = pd.json_normalize(data2['FW'])
radar_fw.loc[:,'position'] = 'FW'
radar_fwr = pd.json_normalize(data2['FWR'])
radar_fwr.loc[:,'position'] = 'FWR'
radar_fwl = pd.json_normalize(data2['FWL'])
radar_fwl.loc[:,'position'] = 'FWL'
radar_mc = pd.json_normalize(data2['MC'])
radar_mc.loc[:,'position'] = 'MC'
radar_mr = pd.json_normalize(data2['MR'])
radar_mr.loc[:,'position'] = 'MR'
radar_ml = pd.json_normalize(data2['ML'])
radar_ml.loc[:,'position'] = 'ML'
radar_sub = pd.json_normalize(data2['Sub'])
radar_sub.loc[:,'position'] = 'Sub'
radar_data = pd.concat([radar_amc,radar_aml,radar_amr,radar_dmc,radar_fw,radar_fwr,radar_fwl,
                       radar_mc,radar_mr,radar_ml,radar_sub])
first_column = radar_data.pop('position') 
radar_data.insert(0, 'position', first_column) 
radar_data

,position,goals.min,goals.max,goals.avg,xG.min,xG.max,xG.avg,shots.min,shots.max,shots.avg,...,xA.avg,key_passes.min,key_passes.max,key_passes.avg,xGChain.min,xGChain.max,xGChain.avg,xGBuildup.min,xGBuildup.max,xGBuildup.avg
0,AMC,0.0003,0.0062,0.0023,0.000660,0.004983,0.002192,0.0111,0.0425,0.0218,...,0.001843,0.0090,0.0451,0.0186,0.002750,0.010071,0.004880,0.001072,0.006206,0.002221
0,AML,0.0008,0.0047,0.0021,0.001378,0.005330,0.002159,0.0171,0.0471,0.0219,...,0.001716,0.0093,0.0302,0.0158,0.002872,0.010276,0.004629,0.000892,0.005197,0.001892
0,AMR,0.0007,0.0055,0.0020,0.000999,0.004492,0.001965,0.0123,0.0362,0.0201,...,0.001813,0.0121,0.0365,0.0165,0.003828,0.009059,0.004554,0.001096,0.004356,0.001925
0,DMC,0.0000,0.0044,0.0006,0.000050,0.003600,0.000667,0.0016,0.0264,0.0103,...,0.000639,0.0022,0.0281,0.0087,0.000955,0.008132,0.003192,0.000889,0.006581,0.002538
0,FW,0.0011,0.0126,0.0042,0.001728,0.012082,0.004155,0.0150,0.0737,0.0280,...,0.001316,0.0036,0.0309,0.0120,0.002727,0.016914,0.005338,0.000243,0.006713,0.001318
0,FWR,0.0021,0.0112,0.0031,0.001775,0.009357,0.002976,0.0216,0.0562,0.0255,...,0.002069,0.0187,0.0349,0.0179,0.004310,0.015081,0.005801,0.001854,0.005891,0.002327
0,FWL,0.0012,0.0118,0.0031,0.001742,0.010550,0.002925,0.0188,0.0693,0.0268,...,0.002104,0.0146,0.0312,0.0179,0.004491,0.014931,0.005866,0.001656,0.005858,0.002326
0,MC,0.0000,0.0052,0.0009,0.000033,0.003700,0.000922,0.0012,0.0354,0.0124,...,0.000968,0.0021,0.0377,0.0115,0.000854,0.012126,0.003672,0.000642,0.008059,0.002569
0,MR,0.0007,0.0033,0.0014,0.000812,0.002948,0.001464,0.0082,0.0262,0.0162,...,0.001528,0.0092,0.0195,0.0146,0.002675,0.005248,0.003840,0.001035,0.003009,0.001810
0,ML,0.0003,0.0032,0.0015,0.000583,0.002587,0.001497,0.0101,0.0261,0.0172,...,0.001521,0.0090,0.0220,0.0149,0.002416,0.006683,0.003775,0.000987,0.003597,0.001802


In [26]:
fw_radar = pd.DataFrame(data2['FW']).T
fw_radar

,min,max,avg
goals,0.001100,0.012600,0.004200
xG,0.001728,0.012082,0.004155
shots,0.015000,0.073700,0.028000
assists,0.000000,0.004800,0.001400
xA,0.000264,0.005382,0.001316
key_passes,0.003600,0.030900,0.012000
xGChain,0.002727,0.016914,0.005338
xGBuildup,0.000243,0.006713,0.001318


In [27]:
strings3 = scripts[3].string
ind_start3 = strings3.index("('")+2 
ind_end3 = strings3.index("')") 
json_data3 = strings3[ind_start3:ind_end3] 
json_data3 = json_data3.encode('utf8').decode('unicode_escape')
data3 = json.loads(json_data3)
shots_df = pd.DataFrame(data3)
shots_df

,id,minute,result,X,Y,xG,player,h_a,player_id,situation,season,shotType,match_id,h_team,a_team,h_goals,a_goals,date,player_assisted,lastAction
0,41640,79,BlockedShot,0.730999984741211,0.5059999847412109,0.031210700049996376,Bruno Fernandes,h,1228,OpenPlay,2014,RightFoot,5158,Udinese,Empoli,2,0,2014-08-31 19:45:00,Cyril Théréau,Pass
1,41641,82,MissedShots,0.9569999694824218,0.6170000076293946,0.11918000131845474,Bruno Fernandes,h,1228,OpenPlay,2014,RightFoot,5158,Udinese,Empoli,2,0,2014-08-31 19:45:00,Antonio Di Natale,Chipped
2,41643,86,SavedShot,0.77,0.5170000076293946,0.037953298538923264,Bruno Fernandes,h,1228,OpenPlay,2014,RightFoot,5158,Udinese,Empoli,2,0,2014-08-31 19:45:00,Panagiotis Kone,Pass
3,41807,47,BlockedShot,0.759000015258789,0.664000015258789,0.01848829910159111,Bruno Fernandes,a,1228,OpenPlay,2014,RightFoot,5110,Juventus,Udinese,2,0,2014-09-13 19:45:00,Antonio Di Natale,LayOff
4,42170,42,BlockedShot,0.754000015258789,0.534000015258789,0.02486089989542961,Bruno Fernandes,h,1228,OpenPlay,2014,LeftFoot,5125,Udinese,Napoli,1,0,2014-09-21 17:00:00,Emmanuel Badu,Pass
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
563,565249,70,SavedShot,0.7930000305175782,0.64,0.026381036266684532,Bruno Fernandes,h,1228,OpenPlay,2023,RightFoot,22152,Manchester United,Fulham,1,2,2024-02-24 15:00:00,Marcus Rashford,TakeOn
564,565250,71,BlockedShot,0.9540000152587891,0.29399999618530276,0.019395293667912483,Bruno Fernandes,h,1228,FromCorner,2023,RightFoot,22152,Manchester United,Fulham,1,2,2024-02-24 15:00:00,None,Aerial
565,565255,88,SavedShot,0.8640000152587891,0.7130000305175781,0.029268236830830574,Bruno Fernandes,h,1228,OpenPlay,2023,RightFoot,22152,Manchester United,Fulham,1,2,2024-02-24 15:00:00,Harry Maguire,Cross
566,565258,91,SavedShot,0.7240000152587891,0.5,0.016553550958633423,Bruno Fernandes,h,1228,OpenPlay,2023,LeftFoot,22152,Manchester United,Fulham,1,2,2024-02-24 15:00:00,Scott McTominay,Pass


In [28]:
strings4 = scripts[4].string
# Getting rid of unnecessary characters from json data
ind_start4 = strings4.index("('")+2 
ind_end4 = strings4.index("')") 
json_data4 = strings4[ind_start4:ind_end4] 
json_data4 = json_data4.encode('utf8').decode('unicode_escape')
data4= json.loads(json_data4)
match_data = pd.DataFrame(data4)
match_data

,goals,shots,xG,time,position,h_team,a_team,h_goals,a_goals,date,id,season,roster_id,xA,assists,key_passes,npg,npxG,xGChain,xGBuildup
0,0,0,0,90,ML,Manchester City,Manchester United,3,1,2024-03-03,22160,2023,645757,0.15376435220241547,1,2,0,0,0.15376435220241547,0
1,0,5,0.10947232693433762,90,AMC,Manchester United,Fulham,1,2,2024-02-24,22152,2023,643274,0.0404401533305645,0,2,0,0.10947232693433762,0.2445642650127411,0.1808682680130005
2,0,5,0.8214424848556519,90,AMC,Luton,Manchester United,1,2,2024-02-18,22139,2023,642556,0.11807581782341003,0,2,0,0.8214424848556519,1.6725761890411377,1.4498051404953003
3,0,0,0,90,AMC,Aston Villa,Manchester United,1,2,2024-02-11,22125,2023,640948,0.78354811668396,0,8,0,0,1.1459511518478394,0.47567039728164673
4,0,2,0.11638087034225464,90,AMC,Manchester United,West Ham,3,0,2024-02-04,22122,2023,639077,0.10938521474599838,1,3,0,0.11638087034225464,0.37612348794937134,0.21940374374389648
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
240,1,2,0.09847020357847214,74,Sub,Udinese,Cesena,1,1,2014-10-05,5074,2014,48156,0.16920700669288635,0,1,1,0.09847020357847214,0.2905159890651703,0.19204600155353546
241,0,0,0,57,Sub,Lazio,Udinese,0,1,2014-09-25,5138,2014,47526,0.09006520360708237,0,1,0,0,0.09006520360708237,0.09006520360708237
242,0,1,0.02486089989542961,70,AMC,Udinese,Napoli,1,0,2014-09-21,5125,2014,47314,0,0,0,0,0.02486089989542961,0.02486089989542961,0
243,0,1,0.01848829910159111,44,Sub,Juventus,Udinese,2,0,2014-09-13,5110,2014,46940,0.03042370080947876,0,2,0,0.01848829910159111,0.04891189932823181,0.017866799607872963
